In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz

--2021-11-21 05:23:51--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8446274598 (7.9G) [application/x-gzip]
Saving to: ‘msmarco-docs.tsv.gz’

msmarco-docs.tsv.gz 100%[===================>]   7.87G  1.79MB/s    in 73m 54s 

2021-11-21 06:37:45 (1.82 MB/s) - ‘msmarco-docs.tsv.gz’ saved [8446274598/8446274598]

--2021-11-21 06:37:45--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6457962 (6.2M) [application/x-gzip]
Saving to: ‘msmar

In [ ]:
!gzip -d msmarco-docs.tsv.gz
!gzip -d msmarco-doctrain-queries.tsv.gz
!gzip -d msmarco-doctrain-top100.gz

In [ ]:
pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 132 kB 5.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [ ]:
queries_train=pd.read_table('msmarco-doctrain-queries.tsv',header=None)
queries_train.columns=['qid','query']
print('Shape=>',queries_train.shape)
queries_train.head()

Shape=> (367013, 2)


,qid,query
0,1185869,)what was the immediate impact of the success ...
1,1185868,_________ justice is designed to repair the ha...
2,1183785,elegxo meaning
3,645590,what does physical medicine do
4,186154,feeding rice cereal how many times per day


In [ ]:
queries=queries_train.sample(n=10000,random_state=42).reset_index(drop=True)
print('Shape=>',queries.shape)
queries.head()

Shape=> (10000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# Creating Training Set of Queries
training_queries=queries.iloc[:5000]
print('Shape=>',training_queries.shape)
training_queries.head()

Shape=> (5000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


In [ ]:
# Creating Testing Set of Queries
testing_queries=queries.iloc[5000:]
print('Shape=>',testing_queries.shape)
testing_queries.head()

Shape=> (5000, 2)


,qid,query
5000,1163452,what cut of beef do you use for beef fajita
5001,814996,what is the definition of fodder?
5002,792241,what is rom used for in computers
5003,320763,how much is a chef paid well
5004,165684,does drinking cinnamon tea help blood sugar le...


In [ ]:
train_top100=pd.read_table('/content/msmarco-doctrain-top100',delimiter=' ',header=None)
train_top100.columns=['qid','Q0','docid','rank','score','runstring']
print('Shape=>',train_top100.shape)
train_top100.head()

Shape=> (36701116, 6)


,qid,Q0,docid,rank,score,runstring
0,1185869,Q0,D59221,1,-4.80433,IndriQueryLikelihood
1,1185869,Q0,D59220,2,-4.92127,IndriQueryLikelihood
2,1185869,Q0,D2192591,3,-5.05215,IndriQueryLikelihood
3,1185869,Q0,D2777518,4,-5.05486,IndriQueryLikelihood
4,1185869,Q0,D2371978,5,-5.07048,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for training
training_ranked100=train_top100[train_top100['qid'].isin(training_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',training_ranked100.shape)
training_ranked100.head()

Shape=> (500000, 6)


,qid,Q0,docid,rank,score,runstring
0,130335,Q0,D3035405,1,-4.56299,IndriQueryLikelihood
1,130335,Q0,D730490,2,-4.59616,IndriQueryLikelihood
2,130335,Q0,D566839,3,-4.71841,IndriQueryLikelihood
3,130335,Q0,D256663,4,-4.91729,IndriQueryLikelihood
4,130335,Q0,D825767,5,-4.96762,IndriQueryLikelihood


In [ ]:
# Reducing train_top100 for testing
testing_ranked100=train_top100[train_top100['qid'].isin(testing_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',testing_ranked100.shape)
testing_ranked100.head()

Shape=> (500000, 6)


,qid,Q0,docid,rank,score,runstring
0,756195,Q0,D2913720,1,-4.49350,IndriQueryLikelihood
1,756195,Q0,D2705475,2,-4.65961,IndriQueryLikelihood
2,756195,Q0,D2950170,3,-4.83030,IndriQueryLikelihood
3,756195,Q0,D365575,4,-4.84019,IndriQueryLikelihood
4,756195,Q0,D577816,5,-4.86953,IndriQueryLikelihood


In [ ]:
# Labelling Top 10 as 1 and last 10 as 0
rel=list(range(1,11))
nonrel=list(range(91,101))
training_ranked100['rel']=training_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))
testing_ranked100['rel']=testing_ranked100['rank'].apply(lambda x: 1 if x in rel else ( 0 if x in nonrel else np.nan))

# Result set for Training
training_result=training_ranked100.dropna()
training_result['rel']=training_result['rel'].astype(int)
print('Shape=>',training_result.shape)
training_result.head()

Shape=> (100000, 7)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,qid,Q0,docid,rank,score,runstring,rel
0,130335,Q0,D3035405,1,-4.56299,IndriQueryLikelihood,1
1,130335,Q0,D730490,2,-4.59616,IndriQueryLikelihood,1
2,130335,Q0,D566839,3,-4.71841,IndriQueryLikelihood,1
3,130335,Q0,D256663,4,-4.91729,IndriQueryLikelihood,1
4,130335,Q0,D825767,5,-4.96762,IndriQueryLikelihood,1


In [ ]:
# Result set for Testing
testing_result=testing_ranked100.dropna()
testing_result['rel']=testing_result['rel'].astype(int)
print('Shape=>',testing_result.shape)
testing_result.head()

Shape=> (100000, 7)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,qid,Q0,docid,rank,score,runstring,rel
0,756195,Q0,D2913720,1,-4.49350,IndriQueryLikelihood,1
1,756195,Q0,D2705475,2,-4.65961,IndriQueryLikelihood,1
2,756195,Q0,D2950170,3,-4.83030,IndriQueryLikelihood,1
3,756195,Q0,D365575,4,-4.84019,IndriQueryLikelihood,1
4,756195,Q0,D577816,5,-4.86953,IndriQueryLikelihood,1


In [ ]:

df=pd.read_csv('msmarco-docs.tsv',header=None,sep="/t",engine=python)
df.columns=['docid','url','title','body']
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
def create_corpus(result):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  print('Number of Rows=>',len(corpus))
  return corpus

training_corpus=create_corpus(training_result)
training_corpus.head()

Number of Rows=> 91572


,docid,title,body
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...
1,D1270076,How long does it take for a mourning doves egg...,How long does it take for a mourning doves egg...
2,D2442854,Top 30 Doctor insights on: What Are The Differ...,Top 30 Doctor insights on: What Are The Differ...
3,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...
4,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...


In [ ]:
testing_corpus=create_corpus(testing_result)
testing_corpus.head()

Number of Rows=> 91725


,docid,title,body
0,D2814506,How much money does a retail sales person earn?,Answers.com ® Wiki Answers ® Categories Busine...
1,D2290875,What does it mean when a dog's nose is hot?,Answers.com ® Wiki Answers ® Categories Animal...
2,D2146220,Secure Digital,"Secure Digital From Wikipedia, the free encycl..."
3,D3239690,How To Substitute Gluten Free Flour in Recipes?,How To Substitute Gluten Free Flour in Recipes...
4,D2120243,What is the largest object in our solar system?,Answers.com ® Wiki Answers ® Categories Scienc...


In [ ]:
temp_doc=training_corpus.sample(n=1)
print('Title=>',temp_doc.title.values)
print('Body:\n',temp_doc.body.values)


Title=> dask.array<values, shape=(nan,), dtype=object, chunksize=(nan,), chunktype=numpy.ndarray>
Body:
 dask.array<values, shape=(nan,), dtype=object, chunksize=(nan,), chunktype=numpy.ndarray>


/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:1274: UserWarning: sample does not support the number of sampled items parameter, 'n'. Please use the 'frac' parameter instead.
  warnings.warn(msg)


In [ ]:
training_corpus.head(50)

,docid,title,body
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...
1,D1270076,How long does it take for a mourning doves egg...,How long does it take for a mourning doves egg...
2,D2442854,Top 30 Doctor insights on: What Are The Differ...,Top 30 Doctor insights on: What Are The Differ...
3,D2378859,How many decibels can a human hear in?,Answers.com ® Wiki Answers ® Categories Scienc...
4,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
5,D2337005,Peripheral Vascular Surgery-Chapter 32,165 terms alexandriamartinez19Peripheral Vascu...
6,D2727318,Laser Tag,Wiki Answers ® Categories Entertainment & Arts...
7,D297612,"Fair Oaks, CA County Of Sacramento","Home Fair Oaks, CA County Of Sacramento Fair O..."
8,D2078142,What are all the literary devices? List them p...,Education & Reference Homework Help What are a...
9,D1344056,Warminster Court,Warminster Court Home Law Enforcement & Milita...


In [ ]:
for i,v in enumerate(training_queries['query'].sample(10)):
    print(i,'=>',v)

0 => what subjects are covered on the ged test
1 => what xo means the weeknd
2 => how to rewarm baked potatoes
3 => what to eat to treat bursitis
4 => how do you get to salina islanda
5 => what are opportunistic infections
6 => where is edgewood, ny located?
7 => what is considered a dangerously low blood pressure?
8 => is roundup safe for animals
9 => what is the difference between health care proxy and durable power of attorney


In [ ]:
import re

# Lowercasing the text
training_corpus['cleaned']=training_corpus['body'].apply(lambda x:x.lower())
testing_corpus['cleaned']=testing_corpus['body'].apply(lambda x:x.lower())

# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions
training_corpus['cleaned']=training_corpus['cleaned'].apply(lambda x:expand_contractions(x))
testing_corpus['cleaned']=testing_corpus['cleaned'].apply(lambda x:expand_contractions(x))

/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.7/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('cleaned', 'object'))

  warnings.warn(meta_